# PLEASE NOTE: Please run this notebook OUTSIDE a Spark notebook as it should run in a plain Default Python 3.6 Free Environment

This is the last assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you should update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

The purpose of this assignment is to learn how feature engineering boosts model performance. You will apply Discrete Fourier Transformation on the accelerometer sensor time series and therefore transforming the dataset from the time to the frequency domain. 

After that, you’ll use a classification algorithm of your choice to create a model and submit the new predictions to the grader. Done.



In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')
    


In [ ]:
!pip install pyspark==2.4.5

In [ ]:
!pip install https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true

In [3]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="rdd")

22/02/03 01:44:29 WARN Utils: Your hostname, IFernandes resolves to a loopback address: 127.0.1.1; using 192.168.249.76 instead (on interface eth0)
22/02/03 01:44:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/03 01:44:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/03 01:44:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()


So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.3.0 (as of 20th March'19) Please use the code block below to check if you are already on 1.3.0 or higher. 1.3 contains a necessary fix, that's we are running against the SNAPSHOT


In [ ]:
!mkdir -p /home/dsxuser/work/systemml

In [5]:
from systemml import MLContext, dml
ml = MLContext(spark)
ml.setConfigProperty("sysml.localtmpdir", "mkdir /home/dsxuser/work/systemml")
print(ml.version())
    
if not ml.version() == '1.3.0-SNAPSHOT':
    raise ValueError('please upgrade to SystemML 1.3.0, or restart your Kernel (Kernel->Restart & Clear Output)')


Welcome to Apache SystemML!
Version 1.3.0-SNAPSHOT
1.3.0-SNAPSHOT


In [ ]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
!mv shake.parquet?raw=true shake.parquet

Now it’s time to read the sensor data and create a temporary query table.

In [6]:
df=spark.read.parquet('shake.parquet')

In [7]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [15]:
!pip install pixiedust

In [ ]:
import pixiedust
display(df)

CLASS,SENSORID,X,Y,Z
2,12345678,0.0,0.0,0.0
2,qqqqqqqq,-0.45,-0.45,-0.45
2,12345678,-0.01,-0.01,-0.01
2,aUniqueID,0.08,0.08,0.08
2,gholi,-14.45,-14.45,-14.45
2,bcbcbcbc,-6.18,-6.18,-6.18
2,aaaaaaaa,0.0,0.0,0.0
2,aaaaaaaa,0.0,0.0,0.0
2,aaaaaaaa,0.0,0.0,0.0
2,aUniqueID,-0.39,-0.39,-0.39


In [9]:
df.createOrReplaceTempView("df")

We’ll use Apache SystemML to implement Discrete Fourier Transformation. This way all computation continues to happen on the Apache Spark cluster for advanced scalability and performance.

As you’ve learned from the lecture, implementing Discrete Fourier Transformation in a linear algebra programming language is simple. Apache SystemML DML is such a language and as you can see the implementation is straightforward and doesn’t differ too much from the mathematical definition (Just note that the sum operator has been swapped with a vector dot product using the %*% syntax borrowed from R
):

<img style="float: left;" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1af0a78dc50bbf118ab6bd4c4dcc3c4ff8502223">



In [10]:
dml_script = '''
PI = 3.141592654
N = nrow(signal)

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = (n %*% t(k))*(2*PI/N)

Xa = cos(M) %*% signal
Xb = sin(M) %*% signal

DFT = cbind(Xa, Xb)
'''

Now it’s time to create a function which takes a single row Apache Spark data frame as argument (the one containing the accelerometer measurement time series for one axis) and returns the Fourier transformation of it. In addition, we are adding an index column for later joining all axis together and renaming the columns to appropriate names. The result of this function is an Apache Spark DataFrame containing the Fourier Transformation of its input in two columns. 


In [11]:
from pyspark.sql.functions import monotonically_increasing_id

def dft_systemml(signal,name):
    prog = dml(dml_script).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toDF() 
     
         #rename default column names
        .selectExpr('C1 as %sa' % (name), 'C2 as %sb' % (name)) 
     
         #add unique ID per row for later joining
        .withColumn("id", monotonically_increasing_id())
    )
        




Now it’s time to create individual DataFrames containing only a subset of the data. We filter simultaneously for accelerometer each sensor axis and one for each class. This means you’ll get 6 DataFrames. Please implement this using the relational API of DataFrames or SparkSQL. Please use class 1 and 2 and not 0 and 1. <h1><span style="color:red">Please make sure that each DataFrame has only ONE colum (only the measurement, eg. not CLASS column)</span></h1>


In [18]:
x0 = spark.sql("SELECT X FROM df where CLASS = 1")
y0 =  spark.sql("SELECT Y FROM df where CLASS = 1")
z0 =  spark.sql("SELECT Z FROM df where CLASS = 1")
x1 =  spark.sql("SELECT X FROM df where CLASS = 2")
y1 =  spark.sql("SELECT Y FROM df where CLASS = 2")
z1 =  spark.sql("SELECT Z FROM df where CLASS = 2")

Since we’ve created this cool DFT function before, we can just call it for each of the 6 DataFrames now. And since the result of this function call is a DataFrame again we can use the pyspark best practice in simply calling methods on it sequentially. So what we are doing is the following:

- Calling DFT for each class and accelerometer sensor axis.
- Joining them together on the ID column. 
- Re-adding a column containing the class index.
- Stacking both Dataframes for each classes together



In [19]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemml(x0,'x') \
    .join(dft_systemml(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))
    
df_class_1 = dft_systemml(x1,'x') \
    .join(dft_systemml(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.7 used for parser compilation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.7 used for parser compilation does not match the current runtime version 4.8
SystemML Statistics:
Total execution time:		0.647 sec.
Number of executed Spark inst:	0.




/home/ivanfernandes/.pyenv/versions/3.8.12/envs/if/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


SystemML Statistics:
Total execution time:		0.086 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.086 sec.
Number of executed Spark inst:	0.


[Stage 26:>                                                         (0 + 8) / 8]
[Stage 26:=============================>                            (4 + 4) / 8]
                                                                                
[Stage 28:>                                                         (0 + 8) / 8]
[Stage 28:===========================================>              (6 + 2) / 8]
SystemML Statistics:
Total execution time:		6.671 sec.
Number of executed Spark inst:	6.

                                                                                

[Stage 34:>                                                         (0 + 8) / 8]
[Stage 34:===========================================>              (6 + 2) / 8]
[Stage 36:>                                                         (0 + 8) / 8

Please create a VectorAssembler which consumes the newly created DFT columns and produces a column “features”


In [20]:
from pyspark.ml.feature import VectorAssembler

In [25]:
vectorAssembler = VectorAssembler(inputCols=["xa","xb","ya","yb","za","zb"], outputCol = "features")
features = vectorAssembler.transform(df_dft)
features.take(10)

[Row(id=0, xa=-0.032325686259876626, xb=0.08552771465661582, ya=-0.032325686259876626, yb=0.08552771465661582, za=-0.032325686259876626, zb=0.08552771465661582, class=0, features=DenseVector([-0.0323, 0.0855, -0.0323, 0.0855, -0.0323, 0.0855])),
 Row(id=7, xa=0.0334907666040125, xb=0.0703175032167542, ya=0.0334907666040125, yb=0.0703175032167542, za=0.0334907666040125, zb=0.0703175032167542, class=0, features=DenseVector([0.0335, 0.0703, 0.0335, 0.0703, 0.0335, 0.0703])),
 Row(id=6, xa=-0.08486126908675795, xb=-0.0218913608396156, ya=-0.08486126908675795, yb=-0.0218913608396156, za=-0.08486126908675795, zb=-0.0218913608396156, class=0, features=DenseVector([-0.0849, -0.0219, -0.0849, -0.0219, -0.0849, -0.0219])),
 Row(id=9, xa=0.005911470409021104, xb=-0.07661295866269147, ya=0.005911470409021104, yb=-0.07661295866269147, za=0.005911470409021104, zb=-0.07661295866269147, class=0, features=DenseVector([0.0059, -0.0766, 0.0059, -0.0766, 0.0059, -0.0766])),
 Row(id=5, xa=-0.00210545483171

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to set the “class” column as target.


In [26]:
from pyspark.ml.classification import GBTClassifier

In [27]:
classifier = GBTClassifier(labelCol = "class")

Let’s train and evaluate…


In [30]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [31]:
model = pipeline.fit(df_dft)

In [32]:
prediction = model.transform(df_dft)

In [33]:
prediction.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|        id|                  xa|                  xb|                  ya|                  yb|                  za|                  zb|class|            features|       rawPrediction|         probability|prediction|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|         0|-0.03232568625987...| 0.08552771465661582|-0.03232568625987...| 0.08552771465661582|-0.03232568625987...| 0.08552771465661582|    0|[-0.0323256862598...|[0.87506135510249...|[0.85196827864295...|       0.0|
|         7|  0.0334907666040125|  0.0703175032167542|  0.0334907666040125|  0.0703175032167542|  0.0334907666040125|  0.070

22/02/03 01:59:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/02/03 01:59:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")
    
binEval.evaluate(prediction) 

0.9981761070017225

If you are happy with the result (I’m happy with > 0.8) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. 


In [35]:
!rm -Rf a2_m4.json

In [36]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')

In [37]:
!rm -f rklib.py
!wget wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2022-02-03 02:00:19--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2022-02-03 02:00:21--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-02-03 02:00:21 (8.09 MB/s) - ‘rklib.py’ saved [2540/2540]

FINISHED --2022-02-03 02:00:21--
Total wall clock time: 2.8s
Downloaded: 1 files, 2.5K in 0s (8.09 MB/s)


In [38]:
from rklib import zipit
zipit('a2_m4.json.zip','a2_m4.json')

In [39]:
!base64 a2_m4.json.zip > a2_m4.json.zip.base64

In [40]:
from rklib import submit
key = "-fBiYHYDEeiR4QqiFhAvkA"
part = "IjtJk"
email = "ivanbfernandes@hotmail.com"
submission_token = "AscWFNhn9oBVobCD"

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, submission_token, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"B8wXV","id":"f_F-qCtuEei_fRLwaVDk3g~B8wXV~s9HGp4SMEey-Wg7-pNEX1Q","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
